## Oligo Search Code

Here is the code for the search feature in my website. 

I have optomized it by comparing different python functions and methods with timeit below. 

The rest of the code in match_oligo/views.py is specific to django and allows access to and manipulation of data.

In [ ]:
import xlrd
oligo_sheet = (xlrd.open_workbook("kh_oligos.xlsx")).sheet_by_index(0)
reference = (open("reference.txt")).read()


def reverse_complement(text):
    text = text[::-1].upper().replace(' ','')
    reverse_complement_text = text.translate(str.maketrans('ACGT','TGCA'))
    return reverse_complement_text


def oligo_search(reference, oligo_sheet):
    oligo_column = 2
    reference = reference.upper().replace(' ','')
    rc_reference = reverse_complement(reference)
    matching_oligos = []

    for i in range(oligo_sheet.nrows):
        oligo = (oligo_sheet.cell_value(
                    rowx=i,
                    colx=oligo_column)
                    .upper().replace(" ",""))
        if i < oligo_sheet.nrows and oligo != "" and ((oligo in reference) or (oligo in rc_reference)):
            matching_oligos.append(oligo_sheet.cell_value(rowx=i, colx=oligo_column-1))

    if not matching_oligos:
        return("No oligos match the reference.")
    else:
        return(matching_oligos)


oligo_search(reference, oligo_sheet)

## Optimizing reverse compliment function.

In [ ]:
open_input = open('pattern_count_input.txt')
text = open_input.read().rstrip().replace(" ", "")
from timeit import timeit
from Bio.Seq import Seq

def reverse_complement_one(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse_complement_string = ''
    for i, nucleotide in enumerate(text):
        basepair = (text[((i+1)*-1)+len(text)])
        if dict_basepair.get(basepair):
            reverse_complement_string += dict_basepair.get(basepair)
    return(reverse_complement_string)
#reverse_complement_one:28.062561758997617


def reverse_complement_two(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse = (text[::-1])
    reverse_complement_string = ''
    for i in reverse:
        if dict_basepair.get(i):
            reverse_complement_string += dict_basepair.get(i)
    return(reverse_complement_string)
#reverse_complement_two:16.897074198001064


def reverse_complement_three(text):
    text = text.upper().replace(' ','')
    dict_basepair = {'G':'C', 'C':'G', 'A':'T', 'T':'A'}
    reverse_complement_string = ''
    for nucleotide in reversed(text):
        if dict_basepair.get(nucleotide):
            reverse_complement_string += dict_basepair.get(nucleotide)
    return (reverse_complement_string)
#reverse_complement_three:16.088491767994128


def reverse_complement_four(text):
    text = text[::-1].upper().replace(' ','')
    reverse_complement_text = text.translate(str.maketrans('ACGT','TGCA'))
    return reverse_complement_text
#reverse_complement_four:4.95408325898461


def reverse_complement_biopython(text):
    sequence = Seq(text)
    reverse_complement_sequence = Seq.reverse_complement(sequence)
    return(reverse_complement_sequence)
#reverse_complement_biopython:17.880276261013933


print ("reverse_complement_one:{}".format(timeit(
                                    "reverse_complement_one(text)",
                                    "from __main__ import reverse_complement_one;"
                                    "text='A@GG GTTAG--TGACCAGCT! AG'")))
print ("reverse_complement_two:{}".format(timeit(
                                    "reverse_complement_two(text)",
                                    "from __main__ import reverse_complement_two;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'")))
print ("reverse_complement_three:{}".format(timeit(
                                    "reverse_complement_three(text)",
                                    "from __main__ import reverse_complement_three;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'")))
print ("reverse_complement_four:{}".format(timeit(
                                    "reverse_complement_four(text)",
                                    "from __main__ import reverse_complement_four;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'")))
print ("reverse_complement_biopython:{}".format(timeit(
                                    "reverse_complement_biopython(text)",
                                    "from __main__ import reverse_complement_biopython;"
                                    "text = 'A@GG GTTAG--TGACCAGCT! AG'")))

## Optimizing search function